In [1]:

import os
from obspy import read
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, clear_output

# Save and load progress
progress_file = "progress.txt"

def save_progress(file_path, file_index):
    with open(progress_file, "w") as f:
        f.write(f"{file_path},{file_index}")

def load_progress():
    try:
        with open(progress_file, "r") as f:
            content = f.read().strip().split(',')
            return content[0], int(content[1]) if len(content) > 1 else 0
    except FileNotFoundError:
        return None, 0

def plot_st(st, time_shift1, time_shift2):
    sns.set(style="ticks")
    colors = sns.color_palette("Set2", len(st))
    for i, tr in enumerate(st):
        tr.stats.calib = 1.0
        fig = tr.plot(
            starttime=tr.stats.starttime + tr.stats.sac.o - time_shift1,
            endtime=tr.stats.starttime + tr.stats.sac.o + time_shift2,
            color=colors[i],
            handle=True,
            figsize=(20, 10),
        )
        
        ax = fig.gca()
        t_o = tr.stats.starttime + tr.stats.sac.o
        t_p_wave = tr.stats.starttime + tr.stats.sac.t1
        t_s_wave = tr.stats.starttime + tr.stats.sac.t2
        ax.axvline(t_o, color="black", linestyle="-", lw=2, label="Origin time")
        ax.axvline(t_p_wave, color="r", linestyle="--", lw=1.5, label="P-wave")
        ax.axvline(t_s_wave, color="b", linestyle="--", lw=1.5, label="S-wave")
        ax.legend()
        display(fig)

def clear_text_output():
    clear_output(wait=True)
    plt.close('all')

def count_sac_files(directory):
    return len([f for f in os.listdir(directory) if f.endswith(".SAC")])

source_dir = "./NAFZ_QC_SAC"
target_dir = "./NAFZ_final_SAC"

if not os.path.exists(target_dir):
    os.makedirs(target_dir)

sac_file_count = 0
last_processed, last_index = load_progress()
print(f"Last processed file: {last_processed}")
print(f"Last processed index: {last_index}")

start_processing = not last_processed
if start_processing:
    print("Starting from the beginning (no valid progress file found)")
else:
    print(f"Will resume from: {last_processed}")

stop_requested = False

files = sorted([f for f in os.listdir(source_dir) if f.endswith(".SAC")])
total_files = len(files)
print(f"Found {total_files} SAC files in source directory")

for file_index, file in enumerate(files[last_index:], last_index + 1):
    clear_text_output()
    file_path = os.path.join(source_dir, file)

    if not start_processing:
        if os.path.normpath(file_path) == os.path.normpath(last_processed):
            start_processing = True
            print(f"Resuming processing from {file_path}")
        else:
            print(f"Skipping {file_path}")
            continue

    print(f"Processing file {file_index}/{total_files}: {file_path}")
    try:
        st = read(file_path)
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        continue
    
    for i, tr in enumerate(st):
        print(f"Processing trace {i+1}/{len(st)}: {tr.stats.station}")
        sns.set(style="whitegrid")
        plot_st(st[i : i + 1], time_shift1=10, time_shift2=90)
        print(f"Station: {tr.stats.station}")
        print(f"Distance: {tr.stats.sac.dist}")
        print(f"Current P-pick: {tr.stats.sac.t1}, S-pick: {tr.stats.sac.t2}")

    action = input("Enter action (s: save, d: delete, stop: stop processing): ").lower()
    if action == "stop":
        print("Stopping the processing as requested.")
        stop_requested = True
        break
    elif action == "s":
        target_path = os.path.join(target_dir, file)
        st.write(target_path, format="SAC")
        sac_file_count += 1
        print(f"File saved: {target_path}")
    elif action == "d":
        print("File skipped.")
    else:
        print("Invalid action. File not saved.")
    
    # Save progress
    
    save_progress(file_path, file_index)
    print(f"Progress saved: {file_path}, Index: {file_index}")

    if stop_requested:
        break

clear_text_output()
if stop_requested:
    print("Processing was stopped by the user.")
else:
    print(
        f"Completed processing and copying SAC files. Total SAC files processed and copied: {sac_file_count}."
    )

# Print the last processed file at the end of the script
last_processed, last_index = load_progress()
print(f"Last processed file at script end: {last_processed}")
print(f"Last processed index at script end: {last_index}")

# Count and print the number of SAC files in the target directory
target_sac_count = count_sac_files(target_dir)
print(f"Number of SAC files in target directory ({target_dir}): {target_sac_count}")


Completed processing and copying SAC files. Total SAC files processed and copied: 889.
Last processed file at script end: ./NAFZ_QC_SAC/2013.262.13.51.56.000.YH.DF11..BHZ.SAC
Last processed index at script end: 7740
Number of SAC files in target directory (./NAFZ_final_SAC): 2752
